In [1]:
pip install pennylane torch qiskit scikit-learn numpy pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.5 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 32.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 40.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 7.1 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 41.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 44.0 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.0
    Uninstalling nump

In [2]:
import pennylane as qml
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd


In [3]:
# Define a quantum device with 2 qubits
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

# Quantum circuit layer
def quantum_layer(inputs, weights):
    for i in range(n_qubits):
        qml.RX(inputs[i], wires=i)  # Encode input using RX gates
        qml.RY(weights[i], wires=i)  # Apply fuzzy layer using RY gates

    # Entanglement layer
    qml.CZ(wires=[0, 1])  # Introduce entanglement
    
    return qml.expval(qml.PauliZ(0))  # Measure output


In [4]:
# Quantum layer with trainable weights
weight_shapes = {"weights": (n_qubits,)}
qnode = qml.QNode(quantum_layer, dev, interface="torch")

# Quantum Torch Layer
class QuantumFuzzyLayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(0.01 * torch.randn(n_qubits))  # Trainable quantum parameters

    def forward(self, x):
        # Cast the output of the quantum layer to float32 to match the classical layer's dtype
        return qnode(x, self.weights).type(torch.float32)

In [5]:
class QFNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.q_layer = QuantumFuzzyLayer()
        self.fc = nn.Linear(1, 1)  # Fully connected classical layer

    def forward(self, x):
        # Process each data point individually in the quantum layer
        # and stack the results for passing to the classical layer
        q_outputs = []
        for data_point in x:  # Assume x.shape is [batch_size, 2]
            q_output = self.q_layer(data_point)
            q_outputs.append(q_output)

        # Stack the quantum layer outputs to shape [batch_size, 1]
        x = torch.stack(q_outputs).unsqueeze(1)  # Ensures shape [batch_size, 1]

        x = self.fc(x)  # Classical layer
        return torch.sigmoid(x)  # Sigmoid for binary classification

In [6]:
pip install pandas numpy scikit-learn nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 22.9 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import numpy as np

# Load dataset (replace 'dataset.csv' with actual file path)
df = pd.read_csv("/workspaces/Dissertation/twitter_training.csv")

# Display the first few rows
print(df.head())


     ID                                             Tweets Sentiment
0  2401  im getting on borderlands and i will murder yo...  Positive
1  2401  I am coming to the borders and I will kill you...  Positive
2  2401  im getting on borderlands and i will kill you ...  Positive
3  2401  im coming on borderlands and i will murder you...  Positive
4  2401  im getting on borderlands 2 and i will murder ...  Positive


In [8]:
df['Sentiment'].unique()

array(['Positive', 'Neutral', 'Negative', 'Irrelevant'], dtype=object)

In [9]:
# Map sentiment labels to binary (1 = Positive, 0 = Negative)
df.loc[:, "Sentiment"] = df["Sentiment"].map({
    "Positive": 1,
    "Neutral": np.nan,  # Drop neutral tweets
    "Irrelevant": np.nan,  # Drop irrelevant tweets
    "Negative": 0
})

# Drop neutral tweets
df = df.dropna()

# Display class distribution
print(df["Sentiment"].value_counts())


Sentiment
0.0    21637
1.0    19628
Name: count, dtype: int64


In [10]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# Download required NLTK resources
nltk.download("stopwords")
nltk.download("punkt")
nltk.download('punkt_tab') # Download the punkt_tab resource

stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()

def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    # Remove special characters, numbers
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    # Tokenization
    words = word_tokenize(text)
    # Remove stopwords
    words = [word for word in words if word not in stop_words]
    # Stemming
    words = [stemmer.stem(word) for word in words]

    return " ".join(words)

# Apply preprocessing to all tweets
df["cleaned_text"] = df["Tweets"].apply(preprocess_text)

# View cleaned text
print(df.head())

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


     ID                                             Tweets Sentiment  \
0  2401  im getting on borderlands and i will murder yo...       1.0   
1  2401  I am coming to the borders and I will kill you...       1.0   
2  2401  im getting on borderlands and i will kill you ...       1.0   
3  2401  im coming on borderlands and i will murder you...       1.0   
4  2401  im getting on borderlands 2 and i will murder ...       1.0   

                cleaned_text  
0   im get borderland murder  
1           come border kill  
2     im get borderland kill  
3  im come borderland murder  
4   im get borderland murder  


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert text into numerical features
vectorizer = TfidfVectorizer(max_features=5000)  # Limit to 5000 most important words
X = vectorizer.fit_transform(df["cleaned_text"])

# Convert to DataFrame for better readability
X_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# Target variable
y = df["Sentiment"]

# Display processed dataset
print(X_df.shape, y.shape)


(41265, 5000) (41265,)


In [12]:
from sklearn.model_selection import train_test_split

# Split dataset into training and testing (80%-20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Train Shape: {X_train.shape}, Test Shape: {X_test.shape}")


Train Shape: (33012, 5000), Test Shape: (8253, 5000)


In [13]:
'''
# Load dataset
df = pd.read_csv('/content/twitter_training.csv')  # Replace with actual dataset file

# Convert sentiment labels to binary (1 for positive, 0 for negative)
df.loc[:, "Sentiment"] = df["Sentiment"].map({
    "Positive": 1,
    "Neutral": np.nan,  # Drop neutral tweets
    "Irrelevant": np.nan,  # Drop irrelevant tweets
    "Negative": 0
})
df = df.dropna()

# Display class distribution
print(df["Sentiment"].value_counts())
'''
# Preprocessing - Convert text to TF-IDF features
vectorizer = TfidfVectorizer(max_features=2)  # Limited to match quantum circuit inputs
X = vectorizer.fit_transform(df["cleaned_text"]).toarray()
y = df["Sentiment"].values

# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
# ... (previous code)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)

# Ensure y_train contains only numerical values (0 or 1)
y_train = torch.tensor(y_train.astype(np.int64), dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test.astype(np.int64), dtype=torch.float32).view(-1, 1)


In [14]:
# Initialize model
model = QFNN()
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()  # Binary cross-entropy loss for classification

# Training loop
epochs = 10
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    # Print progress
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

# Save the trained model
torch.save(model.state_dict(), "qfnn_model.pth")


Epoch 1/10, Loss: 1.0874


: 